In [ ]:
#!pip install pandas_summary
#!pip install waterfallcharts
#!pip install treeinterpreter

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import waterfall_chart

from fastai.imports import *
from fastai.tabular import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_recall_curve, confusion_matrix, recall_score, precision_score
from sklearn.model_selection import cross_val_score
from matplotlib.ticker import FuncFormatter

In [ ]:
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
import pylab as pl
import numpy as np

import warnings                              # Disable some warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
import pandas as pd

df_1= pd.read_parquet("part-00000-c39f1c5c-b6aa-4659-8dd9-965ccc023d43-c000.snappy.parquet")
df_2 =pd.read_parquet("part-00001-912fc2f1-c131-4414-b9d8-bd9d4095ada5-c000.snappy.parquet")
df = pd.concat([df_1, df_2], axis=0, ignore_index=True)
df.tail()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
start_date = pd.to_datetime("2017-01-20")
end_date = pd.to_datetime("2021-01-20")
df['is_president'] = np.where((df['date'] >= start_date) & (df['date'] <= end_date),1,0)
df['is_president'].value_counts()

In [ ]:
df2=pd.read_csv("dataset.csv")
df2

In [ ]:
import pandas as pd

# Convert date columns to datetime format with consistent format
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df2['Date'] = pd.to_datetime(df2['Date'], format='%Y-%m-%d', errors='coerce')

# Remove time component from df1 to match df2 if df2 only has date (not datetime)
df['date'] = df['date'].dt.date
df2['Date'] = df2['Date'].dt.date

# Merge both datasets on the date column
merged_df = pd.merge(df, df2, left_on='date', right_on='Date', how='inner')

# Drop duplicate date column from df2 if necessary
merged_df.drop(columns=['Date'], inplace=True)

merged_df


In [ ]:
merged_df.isnull().sum()

In [ ]:
merged_df["btc_price_increased"]=np.where(merged_df["btc_price_24h_later"]-merged_df["btc_current_price"]>0,1,0)
columns_to_keep=["sentiment_score","btc_current_price","is_president","Value","BTC_Volume","btc_price_increased"]
merged_df=merged_df[columns_to_keep]
merged_df

In [ ]:
target = "btc_price_increased"
merged_df.describe()

In [ ]:
df_model, df_reserved = train_test_split(merged_df, test_size=0.1,random_state=42)
print ("Sample size dataset reserved for prediction: ", df_reserved.shape[0], "records")

In [ ]:
y = df_model[target]

# cross validation (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(df_model, y, test_size=0.2)
print ("Sample size train dataset: ", X_train.shape)
print ("Sample size test dataset: ", X_test.shape)

In [ ]:
# Stepwise function
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.1, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [ ]:
# Removes it from the explanatory set (as well as id)
#irrelevant_columns = ['id', target]
irrelevant_columns = [target]
X_train2=X_train.drop(columns=irrelevant_columns)

for field in X_train2:
    if X_train2[field].isnull().any():
        X_train2=X_train2.drop(labels=[field], axis=1)
        
X_train2

In [ ]:
result = stepwise_selection(X_train2, y_train)
print('resulting features:')
print(result) # relevant variables
type(result)

In [ ]:
# Adds selected features
X_train_stepwise=X_train2
#for item in X_train2.columns:
 #   if item not in result:
  #      X_train_stepwise=X_train_stepwise.drop(labels=[item],axis=1) #removes the non relevant variables

X_train_stepwise

In [ ]:
X_test_stepwise = X_test

#for item in X_test.columns:
 #   if item not in result:
  #      X_test_stepwise=X_test_stepwise.drop(labels=[item],axis=1) #removes the non relevant variables

X_test_stepwise

In [ ]:
# and heatmap to show correlation between explanatory variables
from matplotlib import pyplot as plt        
import seaborn as sns                       

sns.set(font_scale=1.1)
fig, ax = plt.subplots(figsize=(10,10))         
sns.heatmap(X_train_stepwise.corr(), annot=True, fmt=".2f", linewidths=1, ax=ax)

In [ ]:
X_train_stepwise = sm.add_constant(X_train_stepwise)

# statsmodels
import statsmodels.api as sm
logit_model=sm.Logit(y_train,X_train_stepwise)
result=logit_model.fit_regularized()
print(result.summary2())

# print Exp(B) and confidence intervals
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['Lower', 'Upper', 'Odds Ratio']
print ("\nexp(B) & confidence intervals: ")
print (np.exp(conf))

In [ ]:
cutoff = 0.56 # Every probability above 50% is considered a positive

train_preds = result.predict(X_train_stepwise)
train_preds_binary = (train_preds > cutoff).astype(int)  # Convert to binary labels (0 o 1)
train_conf_matrix = confusion_matrix(y_train, train_preds_binary)

train_conf_matrix_percentage = train_conf_matrix.astype('float') / train_conf_matrix.sum() * 100
train_conf_matrix_percentage = np.round(train_conf_matrix_percentage, 2)

print("Train dataset confusion matrix")
print(train_conf_matrix_percentage)

## BAGGING

In [ ]:
merged_df

In [ ]:
#split data into train and test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(merged_df.iloc[:,merged_df.columns != target],
                                                    merged_df[target],
                                                    test_size=0.25, 
                                                    random_state=100)

In [ ]:
print("Train: ", x_train.shape)
print("Test: ", x_test.shape)

In [ ]:
baseRF = RandomForestClassifier(n_estimators = 200,  # The number of trees in the forest
                               random_state = 0, # This parameter allows to replicate results
                               n_jobs = -1, # The number of jobs to run in parallel (-1 means all processors, 1 no parallelism)
                               oob_score = True) # Whether to use out-of-bag samples to estimate the generalization score (increases time)

# The OOB score is an estimate of the generalization error of the model, calculated with the data not used to train it.
# Since each tree is trained on a bootstrapped sample, they all use approximately 2/3 of the data (this is a mathematical consequence 
# of random sampling with replacement). Therefore, for each of the learners we have 1/3 of the dataset as Out Of Bag (OOB) observations 
# which can be used to test the model and estimate its generalisation error. By averaging these OOB error estimates across the trees, 
# RF produces a robust error estimate for the ensemble during training time at almost no additional computational cost.

# other parameters:
# criterion{"gini", "entropy", "log_loss"}, default="gini"
# max_depth: int, default=None
# min_samples_leaf: int or float, default=1 - The minimum number of samples required to be at a leaf node
# min_samples_split: int or float, default=2 - The minimum number of samples required to split an internal node:
# max_features{"sqrt", "log2", None}, int or float, default="sqrt" - The number of features to consider when looking for the best split

baseRF.fit(x_train,y_train)

# predictions
y_pred_test = baseRF.predict(x_test)

print("OOB score: {:.2%}".format(baseRF.oob_score_))

In [ ]:
param_grid = {
    'min_samples_leaf' :[1,3,5], 
    'max_features' : [10,15,20,25,30],
    'criterion' : ['gini','entropy'] #,'log_loss']
}

In [ ]:
import time 
start_time = time.time()

from sklearn.model_selection import GridSearchCV
cvRF = GridSearchCV(estimator=baseRF, param_grid=param_grid, cv=5, scoring='accuracy') 
#cv is the number of cross validation iterations to be performed
cvRF.fit(x_train,y_train)

print ("Completed in {:0.0f} seconds ".format((time.time() - start_time)))

In [ ]:
# shows the best combination of criteria found
best_params = cvRF.best_params_
print("Best parameters:", best_params)

In [ ]:
print("Accuracy: ", '{0:0.2%}'.format(cvRF.best_score_))


## RANDOM FOREST

In [ ]:
# Random Forest
model_RF = RandomForestClassifier(n_estimators = 100, 
                               random_state = 0,
                               max_features = 10, #this parameter makes the difference between simple Bagging and Random Forests
                               n_jobs = -1,
                               oob_score = True,
                             criterion = 'entropy')
                              # min_samples_leaf = 5)
model_RF.fit(x_train,y_train)

# predictions
y_pred_test = model_RF.predict(x_test)

In [ ]:
print("OOB score: {:.2%}".format(model_RF.oob_score_))

In [ ]:
# We can compare Recall and Precision

prob_pred = model_RF.predict_proba(x_test)[:, 1]
thresholds = np.arange(0.0, 1.0, step=0.01)
recall_scores = [metrics.recall_score(y_test, prob_pred > t) for t in thresholds]
precis_scores = [metrics.precision_score(y_test, prob_pred > t) for t in thresholds]
fig, ax = plt.subplots(1, 1)
ax.plot(thresholds, recall_scores, label="Recall @ t")
ax.plot(thresholds, precis_scores, label="Precision @ t")
ax.axvline(0.5, c="gray", linestyle="--", label="Default Threshold")
ax.set_xlabel("Threshold")
ax.set_ylabel("Metric @ Threshold")
ax.set_box_aspect(1)
ax.legend()
plt.show()

In [ ]:
importances = model_RF.feature_importances_

indexes = np.argsort(importances)[::-1]
sorted_imp = importances[indexes]

plt.figure(figsize=(12, 6))
bars = plt.bar(range(len(importances)), importances[indexes], align='center')
plt.xticks(range(len(importances)), np.array(model_RF.feature_names_in_)[indexes], rotation=90)
plt.xlabel("Features")
plt.ylabel("Importance")
plt.xticks(rotation=90, fontsize=8)
plt.title("Relative feature importances (from 0 to 1)")
for i, bar in enumerate(bars):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(),
             f'{sorted_imp[i]:.2f}', ha='center', va='bottom')
plt.show()

In [ ]:
train_scores = []
cv_scores    = []

estimator_range = range(10, 210, 10) #from 5 trees to 100, step 5

for n_estimators in estimator_range:
    model_RF = RandomForestClassifier(
                n_estimators = n_estimators,
                max_features = 15, # the previous optimum
                oob_score    = False,
                n_jobs       = -1, 
                random_state = 0
             )
    
    model_RF.fit(x_train, y_train)
    predictions = model_RF.predict(x_test)
    acc=accuracy_score(y_test, predictions)
    print("% Accuracy of test dataset for {} trees is {:.3f}".format(n_estimators, acc))
    train_scores.append(acc) # storing accuracy from each iteration

    # and we also store the accuracy mean obtained from running a 5-fold validation
    cvscores = cross_val_score(
                estimator = model_RF,
                X         = x_train,
                y         = y_train,
                scoring   = 'accuracy',
                cv        = 5
             )
    cv_scores.append(cvscores.mean()) 

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(estimator_range, train_scores, marker='o', linestyle='-', color='b')
plt.title('Accuracy of Test Dataset vs. Number of Trees')
plt.xlabel('Number of Trees (n_estimators)')
plt.ylabel('Accuracy')
plt.xticks(estimator_range)  # Para mostrar todos los valores en el eje x
plt.grid()
plt.show()

## XGBoost

In [ ]:
# Separate features from response field
X, y = df_model.iloc[:,:-1], df_model.iloc[:,-1]

# Partition dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.5, random_state= 123)

In [ ]:
import xgboost as xgb
from sklearn.metrics import precision_score

# Create DMatrix (recommended for optimization)
data_dmatrix = xgb.DMatrix(data=X, label=y)

# Define XGBoost Classifier
xg_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=20, eval_metric='logloss')



In [ ]:
# Fit the model
xg_clf.fit(X_train, y_train)

# Predict class labels
y_pred = xg_clf.predict(X_test)

# Compute Precision
precision = precision_score(y_test, y_pred)

print("Precision: {:.2%}".format(precision))

In [ ]:
import xgboost as xgb

# Define parameters
params = {
    'objective': 'binary:logistic',  # Binary classification
    'colsample_bytree': 0.3,
    'learning_rate': 0.1,
    'max_depth': 5,
    'alpha': 10,
    'eval_metric': 'precision'  # Metric for precision
}

# Create DMatrix
data_dmatrix = xgb.DMatrix(data=X, label=y)

# Perform Cross Validation with precision as the metric
cv_results = xgb.cv(
    dtrain=data_dmatrix,
    params=params,
    nfold=3,
    num_boost_round=200,
    early_stopping_rounds=5,
    metrics="logloss",  # Use Precision as the evaluation metric
    as_pandas=True,
    seed=123
)

# Display results
cv_results


In [ ]:
gbm_param_grid = {'learning_rate': [0.01, 0.1, 0.2], 'subsample':[0.3,0.4,0.5]} #subsample controls % of data used
gbm = xgb.XGBClassifier()

grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='precision', cv=5, verbose=2)
grid_mse.fit(X, y)

In [ ]:
xgb.plot_importance(xg_clf, importance_type="gain")

# weight: number of times the variable has been used
# gain: average gain whenever variable is used
# cover: how many samples were affected by using each variable